In [48]:
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
import datetime

API_KEY = "PK1W6UGQYV6DJYD9HEP7"
SECRET_KEY = "ndH61mpUAGkK7jNKM8fgl1EvjUQKf0yDofyYP0pe"

client = CryptoHistoricalDataClient()
request_params = CryptoBarsRequest(
  symbol_or_symbols=["BTC/USD", 'ETH/USD', 'USDT/USD', 'XRP/USD', 'BNB/USD', 'SOL/USD', 'USDC/USD', 'TRX/USD', 'DOGE/USD', 'ADA/USD'],
  timeframe=TimeFrame.Day,
  start=datetime.datetime(2020, 7, 1),
  end=datetime.datetime(2025, 7, 1)
)

btc_bars = client.get_crypto_bars(request_params)

df = btc_bars.df

In [49]:
import pandas as pd

ts = df.index.get_level_values('timestamp')
ts = pd.to_datetime(ts)

dow_dummies = pd.get_dummies(ts.day_name(), prefix='is', dtype=bool).reindex(
    columns=['is_Monday', 'is_Tuesday', 'is_Wednesday', 'is_Thursday',
             'is_Friday', 'is_Saturday', 'is_Sunday'], fill_value=False)

def season(month):
    if month in (12, 1, 2):
        return 'winter'
    elif month in (3, 4, 5):
        return 'spring'
    elif month in (6, 7, 8):
        return 'summer'
    else:
        return 'autumn'

season_dummies = pd.get_dummies([season(m) for m in ts.month], prefix='is', dtype=bool).reindex(
    columns=['is_winter', 'is_spring', 'is_summer', 'is_autumn'], fill_value=False)

dow_dummies.index = df.index
season_dummies.index = df.index

df = df.join(dow_dummies).join(season_dummies)

In [50]:
import requests

def get_fear_greed_data():
    url = "https://api.alternative.me/fng/?limit=0&format=json"
    response = requests.get(url)
    data = response.json()['data']
    df = pd.DataFrame(data)
    df['timestamp'] = pd.to_datetime(df['timestamp'].astype(int), unit='s').dt.date  # только дата
    df['fear&greed'] = df['value'].astype(int)
    return df[['timestamp', 'fear&greed']]

fng_df = get_fear_greed_data()

df = df.reset_index()
df['date'] = df['timestamp'].dt.date

df = df.merge(fng_df, how='left', left_on='date', right_on='timestamp')

df = df.drop(columns=['timestamp_y', 'date'])
df = df.rename(columns={'timestamp_x': 'timestamp'})

df = df.set_index(['symbol', 'timestamp'])

In [51]:
df

open       high        low      close  \
symbol  timestamp                                                               
SOL/USD 2021-01-01 06:00:00+00:00     1.6162     1.9156     1.5759     1.8036   
        2021-01-02 06:00:00+00:00     1.8138     2.0000     1.7175     1.9576   
        2021-01-03 06:00:00+00:00     1.9766     2.4173     1.9223     2.3033   
        2021-01-04 06:00:00+00:00     2.2891     2.5000     1.8870     2.1059   
        2021-01-05 06:00:00+00:00     2.1037     2.3979     1.9693     2.0000   
...                                      ...        ...        ...        ...   
ETH/USD 2025-06-26 05:00:00+00:00  2483.0310  2501.8155  2384.1650  2450.4620   
        2025-06-27 05:00:00+00:00  2451.9000  2464.0000  2391.2195  2422.4000   
        2025-06-28 05:00:00+00:00  2422.5400  2449.8000  2420.4100  2430.7180   
        2025-06-29 05:00:00+00:00  2431.1000  2524.8325  2413.9000  2497.0550   
        2025-06-30 05:00:00+00:00  2495.9200  2522.1850  2435.7750  2462.4500   

                                          volume  trade_count         vwap  \
symbol  timestamp                                                            
SOL/USD 2021-01-01 06:00:00+00:00  129522.510000        953.0     1.749652   
        2021-01-02 06:00:00+00:00  110355.400000       1024.0     1.881977   
        2021-01-03 06:00:00+00:00  234503.080000       2153.0     2.148064   
        2021-01-04 06:00:00+00:00  153615.320000       2115.0     2.283026   
        2021-01-05 06:00:00+00:00  259699.560000       1871.0     2.161956   
...                                          ...          ...          ...   
ETH/USD 2025-06-26 05:00:00+00:00      21.056042        127.0  2454.784421   
        2025-06-27 05:00:00+00:00       8.990786        157.0  2426.357495   
        2025-06-28 05:00:00+00:00      33.849724         70.0  2430.049730   
        2025-06-29 05:00:00+00:00       9.612288         60.0  2480.611202   
        2025-06-30 05:00:00+00:00      37.985797        188.0  2483.998447   

                                   is_Monday  is_Tuesday  is_Wednesday  \
symbol  timestamp                                                        
SOL/USD 2021-01-01 06:00:00+00:00      False       False         False   
        2021-01-02 06:00:00+00:00      False       False         False   
        2021-01-03 06:00:00+00:00      False       False         False   
        2021-01-04 06:00:00+00:00       True       False         False   
        2021-01-05 06:00:00+00:00      False        True         False   
...                                      ...         ...           ...   
ETH/USD 2025-06-26 05:00:00+00:00      False       False         False   
        2025-06-27 05:00:00+00:00      False       False         False   
        2025-06-28 05:00:00+00:00      False       False         False   
        2025-06-29 05:00:00+00:00      False       False         False   
        2025-06-30 05:00:00+00:00       True       False         False   

                                   is_Thursday  is_Friday  is_Saturday  \
symbol  timestamp                                                        
SOL/USD 2021-01-01 06:00:00+00:00        False       True        False   
        2021-01-02 06:00:00+00:00        False      False         True   
        2021-01-03 06:00:00+00:00        False      False        False   
        2021-01-04 06:00:00+00:00        False      False        False   
        2021-01-05 06:00:00+00:00        False      False        False   
...                                        ...        ...          ...   
ETH/USD 2025-06-26 05:00:00+00:00         True      False        False   
        2025-06-27 05:00:00+00:00        False       True        False   
        2025-06-28 05:00:00+00:00        False      False         True   
        2025-06-29 05:00:00+00:00        False      False        False   
        2025-06-30 05:00:00+00:00        False      False        False   

                                   is_Sunday 